In [6]:
import pandas as pd
import pandas as pd
import optuna
import sys
sys.path.append('D:\Desktop\Вся прога на питоне\Задача по курсовой')
from functions import to_exel,train_test_val,NN_weight

<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Роман\AppData\Local\Temp\ipykernel_9520\717569148.py:5: SyntaxWarning: invalid escape sequence '\D'
  sys.path.append('D:\Desktop\Вся прога на питоне\Задача по курсовой')


In [7]:
train = pd.read_csv('D:/Desktop/Вся прога на питоне/Задача по курсовой/mtsgrvmgn_trn.csv')
test = pd.read_csv('D:/Desktop/Вся прога на питоне/Задача по курсовой/mtsgrvmgn_tst.csv')
valid = pd.read_csv('D:/Desktop/Вся прога на питоне/Задача по курсовой/mtsgrvmgn_vld.csv')

In [8]:
columns = ['H1_8','H2_8','H3_8']
X_train,X_test,X_valid,y_train,y_test,y_valid = train_test_val(train=train,test=test,valid = valid,columns=columns)


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset,Dataset
import torch.nn as nn

class SingleLayerPerceptron(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SingleLayerPerceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_valid.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_valid.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

batch_size = batch_size
dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)




In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import optuna
from sklearn.metrics import mean_squared_error

# Загрузка данных (предположим, что X_train, y_train, X_val, y_val уже загружены)
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_valid.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_valid.values, dtype=torch.float32)

# Определение модели (однослойный перцептрон с функцией активации)
class SingleLayerPerceptron(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SingleLayerPerceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        return self.fc(x)

# Функция для обучения и валидации модели
def train_model(model, dataloader, val_dataloader, criterion, optimizer, num_epochs, patience):
    train_loss_history = []
    val_loss_history = []
    best_val_loss = float('inf')
    counter = 0

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for X_batch, y_batch in dataloader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        # Валидация
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_val_batch, y_val_batch in val_dataloader:
                val_outputs = model(X_val_batch)
                val_loss += criterion(val_outputs, y_val_batch).item()

        val_loss /= len(val_dataloader)
        train_loss_history.append(epoch_loss / len(dataloader))
        val_loss_history.append(val_loss)

        # Ранняя остановка
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

    return best_val_loss

# Целевая функция для Optuna
def objective(trial):
    # Гиперпараметры для оптимизации
    # learning_rate = trial.suggest_categorical("learning_rate", [0.1, 0.01, 1e-3,1e-4,1e-5])
    # batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    # num_epochs = trial.suggest_int("num_epochs", 50, 300)
    # patience = trial.suggest_int("patience", 10, 50)
    learning_rate = trial.suggest_categorical("learning_rate", [0.001])
    batch_size = trial.suggest_categorical("batch_size", [32])
    num_epochs = trial.suggest_categorical("num_epochs", [50])
    patience = trial.suggest_categorical("patience", [15])
    # Создание DataLoader
    dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Инициализация модели, функции потерь и оптимизатора
    input_dim = X_train_tensor.shape[1]
    output_dim = y_train_tensor.shape[1]
    model = SingleLayerPerceptron(input_dim, output_dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Обучение модели
    best_val_loss = train_model(model, dataloader, val_dataloader, criterion, optimizer, num_epochs, patience)

    return best_val_loss

# Оптимизация с помощью Optuna
study = optuna.create_study(direction="minimize")  # Минимизируем ошибку
study.optimize(objective, n_trials=50, timeout=600)  # 50 trials или 10 минут

# Лучшие параметры
best_params = study.best_params
print("Лучшие параметры:", best_params)

# Обучение модели с лучшими параметрами
best_batch_size = best_params["batch_size"]
best_learning_rate = best_params["learning_rate"]
best_num_epochs = best_params["num_epochs"]
best_patience = best_params["patience"]

dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
dataloader = DataLoader(dataset, batch_size=best_batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)

input_dim = X_train_tensor.shape[1]
output_dim = y_train_tensor.shape[1]
best_model = SingleLayerPerceptron(input_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_learning_rate)

# Обучение лучшей модели
train_model(best_model, dataloader, val_dataloader, criterion, optimizer, best_num_epochs, best_patience)

# Оценка на тестовом наборе (если есть)
best_model.eval()
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
    test_outputs = best_model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor).item()
    print(f"Test Loss: {test_loss:.4f}")